Blast

In [ ]:
def query_map(query : str, w : int) -> dict[str, list[int]]:
    """
    query: a string que procuramos
    w: o tamanho alvo
    Devolve um dicionário
        cuja chave são todas as strings de tamanho w da query e
        cujo valor é uma lista com todos os offsets onde aparecem
    """
    qm = {}                               # Inicia-se um dicionário vazio; 

    for i in range(len(query) - w + 1):   # Inicia-se um for loop para ler cada ocorrência;
        substring = query[i : i + w]      # Inicia-se a substring que vai ser lida em comprimentos de w (iteração i até i + w, ou seja, se w = 3, vai ler da iteração 0 até 0 + 3);
        #print(substring)           
        if substring not in qm:           # Vai ler a substring e procurar se ele não ocorre em qm;
            qm[substring] = [i]           # Se não encontrar repetições da ocorrência, retorna a única ocorrência dela;
        else:
            qm[substring].append(i)       # Se encontrar repetições da ocorrência, ADICIONA as posições do primeiro caracter de cada substring.

    return qm

example = query_map("AATATAT",3)
example

In [ ]:
def hits(qm, seq):
    """
        qm: o que é devolvido ao invocar a função query_map
        seq: a sequência alvo

        Devolve uma lista em que cada elemento é um tuplo com dois valores:
            1. O Offset na query
            2. O offset na seq
            
        snip: valor do dicionário qm
        pos_qm: posição do snip, tal revelado no dicionário qm

        Algoritmo
        Navegar pelos pares chave/valor (snip, posições) do qm
            Descobrir todas as posições (P2) em seq onde o snip se encontra
            Navegar (P1) pela lista de posições
                Navegar por essas posições (P2)
                    Adicionar o par (P1, P2) à lista dos resultados
    """
    results = []
    for snip, pos_qm in qm.items(): # Iteração dos pares key/value do dict qm
        if snip in seq: # Verifica se a substring está presente na sequência 'seq'
            pos_seq = [i for i in range(len(seq) - len(snip) + 1) if seq[i:i+len(snip)] == snip] # Encontra todas as posições na sequência dada onde a substring ocorre
            
            for P1 in pos_qm: # Itera sobre as posições na query (pos_qm)
                for P2 in pos_seq: # Itera sobre as posições em 'seq'
                    results.append((P1, P2)) # Adiciona o par (P1, P2) à lista 'results'
                    
    return results

In [ ]:
def extends_hit(seq, hit, query, w):
    """
    query: a string que procuramos
    seq: a sequência alvo
    hit: Um dos elementos devolvidos pela invocação da função hits
    w: o tamanho_extensão da janela

    Devolve um tuplo com:
      1. Devolve a posição inicial da chave na query
      2. Devolve a posição inicial da chave na seq
      3. O tamanho_extensão do resultado
      4. O nº de matches corretos
    """
    
    # Extrai as posições iniciais da correspondência
    start_query, start_seq = hit[0], hit[1]

    # Mover para frente
    match_forward = 0  # Contador para correspondências para frente
    tamanho_extensão = 0  # Tamanho atual da extensão
    melhor_tamanho = 0  # Melhor tamanho_extensão encontrado
    while 2 * match_forward >= tamanho_extensão and start_query + w + tamanho_extensão < len(query) and start_seq + w + tamanho_extensão < len(seq):
        # Verifica correspondência entre a consulta e a sequência
        if query[start_query + w + tamanho_extensão] == seq[start_seq + w + tamanho_extensão]:
            match_forward += 1
            melhor_tamanho = tamanho_extensão + 1
        tamanho_extensão += 1
    size = w + melhor_tamanho  # Calcula o tamanho_extensão total da correspondência estendida para frente

    # Mover para trás
    tamanho_extensão = 0  # Reinicia o tamanho_extensão para a contagem para trás
    match_backwards = 0  # Contador para correspondências para trás
    melhor_tamanho = 0  # Reinicia o melhor tamanho_extensão encontrado
    while 2 * match_backwards >= tamanho_extensão and start_query > tamanho_extensão and start_seq > tamanho_extensão:
        # Verifica correspondência entre a consulta e a sequência para trás
        if query[start_query - tamanho_extensão - 1] == seq[start_seq - tamanho_extensão - 1]:
            match_backwards += 1
            melhor_tamanho = tamanho_extensão + 1
        tamanho_extensão += 1
    size += melhor_tamanho  # Adiciona o melhor tamanho_extensão encontrado para a extensão para trás

    return (start_query - melhor_tamanho, start_seq - melhor_tamanho, size, w + match_forward + match_backwards)

# Exemplo de utilização
example = extends_hit("AATATGTTATATAATAATATTT", (1, 16), "AATATAT", 3)
print(example)
